In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns
from ast import literal_eval

In [30]:
df = pd.read_csv('dataset_clean.csv',converters={"list_title": literal_eval,
                               "list_body": literal_eval,
                               "Tags_list": literal_eval})
df.head()

,Unnamed: 0,Id,Tags_list,list_title,list_body
0,0,4,[c#],"[convert, decimal, double, c]","[trackbar, form, opacity, decimal, trans, trac..."
1,1,6,"[html, css]","[width, collapse, percentage, width, child, el...","[absolutely, position, div, contain, several, ..."
2,3,9,"[c#, .net]","[calculate, someone, age, base, datetime, type...","[datetime, represent, person, birthday, calcul..."
3,4,11,[c#],"[calculate, relative, time, c]","[specific, datetime, value, display, relative,..."
4,6,13,[html],"[determine, user, timezone]","[standard, web, server, able, determine, user,..."


# Construction du bag of words

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

X = df['list_title'] + df['list_body']
Y = df["Tags_list"]

# Initialize the "CountVectorizer" TFIDF for Full_doc
vectorizer = TfidfVectorizer(analyzer="word",
                             max_df=.6,
                             min_df=0.005,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)

vectorizer.fit(X)
X_tfidf = vectorizer.transform(X)

print("Shape of X for Full_doc: {}".format(X_tfidf.shape))

# Multilabel binarizer for targets


multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(Y)
y_binarized = multilabel_binarizer.transform(Y)

print("Shape of y: {}".format(y_binarized.shape))

Shape of X for Full_doc: (7696, 1145)
Shape of y: (7696, 50)


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binarized,
                                                    test_size=0.2, random_state=8)

In [33]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(X)

# Create Corpus
texts = X

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [34]:
from pprint import pprint
import gensim

# number of topics
num_topics = 20

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"application" + 0.007*"net" + 0.007*"run" + 0.007*"user" + '
  '0.006*"project" + 0.006*"write" + 0.006*"function" + 0.005*"class" + '
  '0.005*"framework" + 0.005*"test"'),
 (1,
  '0.010*"application" + 0.009*"class" + 0.008*"c" + 0.007*"server" + '
  '0.007*"string" + 0.006*"value" + 0.005*"store" + 0.005*"call" + '
  '0.005*"list" + 0.004*"problem"'),
 (2,
  '0.010*"c" + 0.007*"project" + 0.006*"object" + 0.006*"table" + 0.006*"new" '
  '+ 0.006*"class" + 0.005*"run" + 0.005*"text" + 0.005*"select" + '
  '0.004*"error"'),
 (3,
  '0.009*"problem" + 0.008*"server" + 0.007*"data" + 0.007*"test" + '
  '0.006*"call" + 0.006*"sql" + 0.005*"table" + 0.005*"control" + 0.005*"run" '
  '+ 0.005*"method"'),
 (4,
  '0.016*"c" + 0.008*"server" + 0.008*"class" + 0.007*"data" + '
  '0.007*"database" + 0.006*"sql" + 0.006*"function" + 0.005*"application" + '
  '0.005*"best" + 0.005*"type"'),
 (5,
  '0.010*"test" + 0.007*"object" + 0.007*"c" + 0.006*"java" + 0.006*"well" + '
  '0.005*"

In [35]:
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
from IPython.core.display import display, HTML
pyLDAvis.enable_notebook()


%matplotlib inline

display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

gensimvis.prepare(lda_model, corpus, id2word)

C:\Users\Lucas\Documents\formation_ML_OCR\projet_5\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16     0.008406  0.005581       1        1  8.853528
19    -0.004073  0.006522       2        1  7.322852
0      0.024328  0.009664       3        1  6.922763
12     0.029097 -0.005561       4        1  6.822522
4      0.017917  0.006119       5        1  6.244159
15     0.027064 -0.027081       6        1  6.133036
2      0.015428 -0.025154       7        1  5.271272
5      0.002899  0.005407       8        1  5.172515
18    -0.005092 -0.014846       9        1  5.138254
1      0.006745  0.006340      10        1  5.086697
7      0.025391 -0.008630      11        1  5.007820
6      0.002367 -0.003932      12        1  4.862985
3     -0.007320  0.012780      13        1  4.587126
11     0.002038  0.007714      14        1  4.008917
9      0.005450 -0.021285      15        1  3.604899
13     0.009085  0.004452      16        1  3.316148
8     -0.013389  0.061174      17        1  3.209728
10    -0.026576  0.001699      18        1  3.041597
17    -0.068590 -0.054967      19        1  2.841280
14    -0.051177  0.034007      20        1  2.551903, topic_info=         Term         Freq        Total Category  logprob  loglift
74     string  1618.000000  1618.000000  Default  30.0000  30.0000
265    object  1452.000000  1452.000000  Default  29.0000  29.0000
716       xml   676.000000   676.000000  Default  28.0000  28.0000
2           c  2841.000000  2841.000000  Default  27.0000  27.0000
124      data  1779.000000  1779.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
331       sql    30.707658  1449.532260  Topic20  -5.7798  -0.1862
186    number    27.444834   781.223837  Topic20  -5.8921   0.3196
310  function    28.593520  1257.451348  Topic20  -5.8511  -0.1153
64       time    28.515526  1268.449406  Topic20  -5.8539  -0.1268
7       error    28.269444  1149.033096  Topic20  -5.8625  -0.0366

[1697 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
4841       1  0.039028                  _
4841       2  0.039028                  _
4841       3  0.143102                  _
4841       4  0.013009                  _
4841       5  0.039028                  _
...      ...       ...                ...
12875     18  0.050179                xsl
12875     20  0.025090                xsl
12683     16  0.782713            ycenter
22219     20  0.779021  yetanotherdataset
24808      8  0.699807           zend_pdf

[6603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 20, 1, 13, 5, 16, 3, 6, 19, 2, 8, 7, 4, 12, 10, 14, 9, 11, 18, 15])